In [1]:
# Import the modules
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [16]:
# Read in the CSV file as a Pandas DataFrame
sales_df = pd.read_csv(
    Path("Resources/US_Regional_Sales_Data.csv"),
    index_col="OrderNumber", 
    parse_dates=True, 
    infer_datetime_format=True
)

# Review the DataFrame
sales_df.head()

C:\Users\lasot\AppData\Local\Temp\ipykernel_3392\1859464474.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  sales_df = pd.read_csv(
C:\Users\lasot\AppData\Local\Temp\ipykernel_3392\1859464474.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales_df = pd.read_csv(


,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price
OrderNumber,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,31/12/17,31/5/18,14/6/18,19/6/18,USD,6,15,259,12,5,0.08,"$1,001.18","$1,963.10"
SO - 000102,Online,WARE-NMK1003,31/12/17,31/5/18,22/6/18,2/7/2018,USD,14,20,196,27,3,0.08,"$3,348.66","$3,939.60"
SO - 000103,Distributor,WARE-UHY1004,31/12/17,31/5/18,21/6/18,1/7/2018,USD,21,16,213,16,1,0.05,$781.22,"$1,775.50"
SO - 000104,Wholesale,WARE-NMK1003,31/12/17,31/5/18,2/6/2018,7/6/2018,USD,28,48,107,23,8,0.08,"$1,464.69","$2,324.90"
SO - 000105,Distributor,WARE-NMK1003,10/4/2018,31/5/18,16/6/18,26/6/18,USD,22,49,111,26,8,0.10,"$1,476.14","$1,822.40"


In [17]:
#show data type for each column
sales_df.dtypes

Sales Channel        object
WarehouseCode        object
ProcuredDate         object
OrderDate            object
ShipDate             object
DeliveryDate         object
CurrencyCode         object
_SalesTeamID          int64
_CustomerID           int64
_StoreID              int64
_ProductID            int64
Order Quantity        int64
Discount Applied    float64
Unit Cost            object
Unit Price           object
dtype: object

In [22]:
#change columns ProcuredDate, OrderDate, ShipDate, and DeliveryDate from a format day/month/year to month/day/year

sales_df['ProcuredDate'] = pd.to_datetime(sales_df['ProcuredDate'], dayfirst=True, errors='coerce')
sales_df['OrderDate'] = pd.to_datetime(sales_df['OrderDate'], dayfirst=True, errors='coerce')
sales_df['ShipDate'] = pd.to_datetime(sales_df['ShipDate'], dayfirst=True, errors='coerce')
sales_df['DeliveryDate'] = pd.to_datetime(sales_df['DeliveryDate'], dayfirst=True, errors='coerce')


sales_df['ProcuredDate'] = pd.to_datetime(sales_df['ProcuredDate'], format='%d/%m/%Y')
sales_df['OrderDate'] = pd.to_datetime(sales_df['OrderDate'], format='%d/%m/%Y')
sales_df['ShipDate'] = pd.to_datetime(sales_df['ShipDate'], format='%d/%m/%Y')
sales_df['DeliveryDate'] = pd.to_datetime(sales_df['DeliveryDate'], format='%d/%m/%Y')



sales_df.head()

C:\Users\lasot\AppData\Local\Temp\ipykernel_3392\1789869082.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales_df['OrderDate'] = pd.to_datetime(sales_df['OrderDate'], dayfirst=True, errors='coerce')
C:\Users\lasot\AppData\Local\Temp\ipykernel_3392\1789869082.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales_df['ShipDate'] = pd.to_datetime(sales_df['ShipDate'], dayfirst=True, errors='coerce')
C:\Users\lasot\AppData\Local\Temp\ipykernel_3392\1789869082.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales_df['DeliveryDate'] = pd.to_datetime(sales_df['Delive

,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price
OrderNumber,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.08,"$1,001.18","$1,963.10"
SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.08,"$3,348.66","$3,939.60"
SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,213,16,1,0.05,$781.22,"$1,775.50"
SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,107,23,8,0.08,"$1,464.69","$2,324.90"
SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,111,26,8,0.10,"$1,476.14","$1,822.40"


In [25]:
#add column "delivery_time" to the dataframe which is the difference between the order date and the delivery date
sales_df["delivery_time"] = sales_df["DeliveryDate"] - sales_df["OrderDate"]
sales_df.head()

,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price,delivery_time
OrderNumber,,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.08,"$1,001.18","$1,963.10",19 days
SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.08,"$3,348.66","$3,939.60",32 days
SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,213,16,1,0.05,$781.22,"$1,775.50",31 days
SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,107,23,8,0.08,"$1,464.69","$2,324.90",7 days
SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,111,26,8,0.10,"$1,476.14","$1,822.40",26 days


In [3]:
# Create a a list to store inertia values
inertia = []

# Create a a list to store the values of k
k = list(range(1, 11))

In [4]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the spread_df DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(spread_df)
    inertia.append(k_model.inertia_)

In [5]:
# Create a Dictionary that holds the list values for k and inertia
elbow_data = {"k": k, "inertia": inertia}

# Create a DataFrame using the elbow_data Dictionary
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,2.835703e+18
1,2,9.903144e+17
2,3,4.999629e+17
3,4,3.062100e+17
4,5,2.101772e+17


In [6]:
# Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

## Perform the following tasks for each of the two most likely values of `k`:

* Define a K-means model using `k` to define the clusters, fit the model, make predictions, and add the prediction values to a copy of the scaled DataFrame and call it `spread_predictions_df`.

* Plot the clusters. The x-axis should reflect the "hi_low_spread", and the y-axis should reflect the "close" price.

In [7]:
# Define the model with the lower value of k clusters
# Use a random_state of 1 to generate the model
model = KMeans(n_clusters=3, random_state=1)

# Fit the model
model.fit(spread_df)

# Make predictions
k_lower = model.predict(spread_df)

# Create a copy of the DataFrame and name it as spread_df_predictions
spread_df_predictions = spread_df.copy()

# Add a class column with the labels to the spread_df_predictions DataFrame
spread_df_predictions['clusters_lower'] = k_lower

In [8]:
# Plot the clusters
spread_df_predictions.hvplot.scatter(
    x="hi_low_spread",
    y="close",
    by="clusters_lower"
).opts(yformatter="%.0f")

:NdOverlay   [clusters_lower]
   :Scatter   [hi_low_spread]   (close)

In [9]:
# Define the model with the higher value of k clusters
# Use a random_state of 1 to generate the model
model = KMeans(n_clusters=4, random_state=1)

# Fit the model
model.fit(spread_df)

# Make predictions
k_higher = model.predict(spread_df)

# Add a class column with the labels to the spread_df_predictions DataFrame
spread_df_predictions['clusters_higher'] = k_higher

In [10]:
# Plot the clusters
spread_df_predictions.hvplot.scatter(
    x="hi_low_spread",
    y="close",
    by="clusters_higher"
).opts(yformatter="%.0f")

:NdOverlay   [clusters_higher]
   :Scatter   [hi_low_spread]   (close)

## Answer the following question

* Considering the plot, what’s the best number of clusters to choose, or value of k? 

From the scatter plots, it's a little hard to tell given the variability and quantity of the data (zooming in helps), but it appears that the optimal value for k, the nubmer of clusters, is 3.